## 기본 예시: 프롬프트 + 모델 + 출력파서
가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것이다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를  물어보는 chain을 생성해보자.

In [3]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "CH01-Basic"
os.environ["LANGSMITH_PROJECT"] = project_name

## 프롬프트 템플릿의 활용
```PromptTemplate```
- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는데 사용되는 템플릿
- 사용법
    - ```template```: 템플릿 문자열. 이 문자열 내에서 중괄호 ```{}```는 변수를 나타낸다.
    - ```input_variables```: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의한다.
```input_variables```
- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트.

In [4]:
from langchain_core.prompts import PromptTemplate

```from_template()``` 메소드를 사용하여 PromptTemplate 객체 생성

In [5]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [6]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [8]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-nano",
)

## Chain 생성
### LCEL(LangChain Expression Language)
![lcel.png](./images/lcel.png)
여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합한다.
```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사. 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그다음  프롬프트 템플릿 출력은 모델로 전달된다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고있는지 이해할 수 있다. 

In [10]:
# prompt를 PromptTemplate 객체로 생성
prompt = PromptTemplate.from_template("{topic}에 대해 {how} 설명해주세요.")
model = ChatOpenAI()
chain = prompt | model

In [11]:
chain

PromptTemplate(input_variables=['how', 'topic'], input_types={}, partial_variables={}, template='{topic}에 대해 {how} 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10f29c190>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10f29cb10>, root_client=<openai.OpenAI object at 0x10f293a90>, root_async_client=<openai.AsyncOpenAI object at 0x10f29c550>, model_kwargs={}, openai_api_key=SecretStr('**********'))

### invoke() 호출
- python 딕셔너리 형태로 입력값을 전달(키:값)
- invoke() 함수 호출 시, 입력값을 전달.

In [12]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정
input = {"topic": "인공지능 모델의 학습 원리", "how": "간단하게"}

In [13]:
# prompt 객체와 model 객체를 파이프() 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달
# 이를 통해 AI 모델이 생성한 메시지를 반환한다.
answer = chain.invoke(input)

In [14]:
print(answer.content)
print(answer.response_metadata)

인공지능 모델의 학습 원리는 데이터를 입력으로 받아들이고, 이를 분석하고 학습하여 원하는 결과를 출력하는 과정입니다. 주어진 데이터를 토대로 모델은 일련의 수학적인 계산을 통해 학습을 진행하고, 입력 데이터와 출력 데이터 사이의 관계를 학습하여 모델을 최적화합니다. 이를 통해 유사한 데이터가 입력되었을 때 모델은 새로운 결과를 예측할 수 있게 됩니다. 이러한 학습 과정을 지도학습, 비지도학습, 강화학습 등의 방법으로 구분할 수 있습니다.
{'token_usage': {'completion_tokens': 195, 'prompt_tokens': 35, 'total_tokens': 230, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CN2rrH6ky8l6sB314JNpdLhBxDI2W', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}


아래는 스트리밍 출력하는 예시

In [15]:
from langchain_teddynote.messages import stream_response
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습은 데이터를 입력으로 받아 원하는 결과를 출력할 수 있도록 모델을 최적화하는 과정입니다. 이를 위해서 먼저 모델 구조를 정의하고 초기화하여 학습을 시작합니다. 

모델은 입력 데이터를 받아 다양한 수학적 연산을 통해 중간 특성을 추출하고, 최종적으로 원하는 결과를 출력합니다. 이때 모델 내부의 매개변수들이 학습되는데, 이는 입력 데이터와 실제 출력값 간의 차이를 최소화하는 방향으로 조절됩니다. 

학습은 다수의 반복 과정을 통해 이루어지는데, 각 반복마다 입력 데이터를 모델에 주입하고 출력값과 실제값의 차이를 계산하여 이를 최소화하는 방향으로 모델의 매개변수를 업데이트합니다. 이러한 과정을 통해 모델은 입력 데이터에 대한 패턴을 학습하고, 새로운 데이터에 대해서도 정확한 예측을 할 수 있도록 최적화됩니다.

### 출력파서(Output Parser)

In [16]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain에 출력파서를 추가한다.

In [17]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성한다
chain = prompt | model | output_parser

In [18]:
# chain 객체의 invoke 메소드를 사용하여 input을 전달한다
input = {"topic": "인공지능 모델의 학습 원리",  "how": "전문적으로"}
chain.invoke(input)

'인공지능 모델의 학습 원리는 크게 지도학습, 비지도학습, 강화학습으로 나눌 수 있습니다.\n\n1. 지도학습(Supervised Learning): 인공지능 모델이 입력 데이터와 정답 데이터가 함께 제공되는 상황에서 학습을 진행하는 방법입니다. 모델은 입력 데이터를 받아들인 후 정답 데이터와 비교하여 오차를 계산하고, 이 오차를 최소화하는 방향으로 모델의 가중치를 조정해나갑니다. 가장 대표적인 지도학습 알고리즘에는 선형 회귀, 로지스틱 회귀, 신경망 등이 있습니다.\n\n2. 비지도학습(Unsupervised Learning): 입력 데이터만을 가지고 학습을 진행하는 방법으로, 정답 데이터가 없는 상황에서 적용됩니다. 모델은 입력 데이터의 특성이나 구조를 파악하고자 노력하며, 주로 데이터의 군집화(clustering)나 차원 축소(dimensionality reduction)에 사용됩니다. 대표적인 비지도학습 알고리즘으로는 K-평균 군집화, PCA(주성분 분석), 오토인코더 등이 있습니다.\n\n3. 강화학습(Reinforcement Learning): 환경과 상호작용하며 학습을 진행하는 방법으로, 보상을 최대화하는 방향으로 행동을 선택하는 방법입니다. 모델은 시행착오를 통해 보상을 최적화하는 방향으로 학습하며, 대표적인 알고리즘으로는 Q-Learning, Deep Q-Network(DQN), 강화학습을 활용한 신경망 등이 있습니다.\n\n이러한 방법들을 조합하여 다양한 분야에 적용되는 인공지능 모델을 학습시키고, 최적의 성능을 얻는 것이 학습의 핵심 원리입니다.'

In [19]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 크게 지도 학습, 비지도 학습, 강화 학습으로 나눌 수 있습니다.

1. 지도 학습 (Supervised Learning):
지도 학습은 입력 데이터와 해당 데이터에 대한 레이블(정답)이 함께 제공되는 방식으로 모델을 학습시키는 방법입니다. 모델은 입력 데이터와 레이블 간의 관계를 학습하여 새로운 입력 데이터에 대한 예측을 할 수 있습니다. 예를 들어, 고양이 사진과 해당 사진이 고양이인지 아닌지에 대한 레이블을 제공하고 모델을 학습시키는 것이 지도 학습의 예시입니다.

2. 비지도 학습 (Unsupervised Learning):
비지도 학습은 레이블이 주어지지 않은 데이터를 활용하여 모델을 학습시키는 방법입니다. 모델은 데이터 간의 패턴이나 특징을 발견하여 데이터를 그룹화하거나 차원을 축소하는 등의 작업을 수행합니다. 예를 들어, 클러스터링이나 차원 축소를 통해 데이터를 이해하는 것이 비지도 학습의 예시입니다.

3. 강화 학습 (Reinforcement Learning):
강화 학습은 에이전트가 환경과 상호작용하면서 보상을 최대화하는 방향으로 학습하는 방법입니다. 에이전트는 특정 상태에서 특정 행동을 취함으로써 보상을 얻게 되고, 이를 토대로 학습하여 미래에 높은 보상을 얻을 수 있는 행동을 선택하도록 합니다. 예를 들어, 게임에서 이길 확률을 높이기 위해 강화 학습을 이용하여 게임을 플레이하는 것이 강화 학습의 예시입니다.

이렇게 다양한 학습 방법을 통해 모델은 입력 데이터의 패턴을 학습하고 문제를 해결할 수 있도록 개선됩니다.

### 템플릿을 변경하여 적용
- 아래의 프롬프트 내용을 얼마든지 변경하여 테스트 해볼 수 있다.
- ```model_name``` 역시 변경하여 테스트가 가능하다 

In [20]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화
model = ChatOpenAI(model_name="gpt-4-turbo")

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

In [21]:
# 체인을 구성한다
chain = prompt | model | output_parser

In [22]:
chain

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\n당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.\n양식은 [FORMAT]을 참고하여 작성해 주세요.\n\n#상황:\n{question}\n\n#FORMAT:\n- 영어 회화:\n- 한글 해석:\n')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10f629ad0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10f644310>, root_client=<openai.OpenAI object at 0x10d9ab350>, root_async_client=<openai.AsyncOpenAI object at 0x10f556390>, model_name='gpt-4-turbo', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [23]:
# 완성된 Chain을 실행하여 답변을 얻는다
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화: 
  "Hi, I'd like to start with some water, please. Could I see the menu? Thank you. I think I'll go for the grilled salmon with a side of steamed vegetables. Does that come with any sauce?"
  
- 한글 해석:
  "안녕하세요, 물부터 시작할게요. 메뉴판 좀 볼 수 있을까요? 감사합니다. 저는 구운 연어에 찐 채소를 곁들인 것으로 주문할게요. 그 음식에 소스가 따로 나오나요?"


In [24]:
# 이번에는 question을 "미국에서 피자 주문"으로 설정하여 실행한다
print(chain.invoke({"question": "미국에서 피자 주문"}))

- 영어 회화:
  A: Hi, I’d like to order a pizza for delivery, please.
  B: Sure, what would you like?
  A: I’d like a large pepperoni pizza with extra cheese.
  B: Would you like any sides or drinks?
  A: Yes, can I have an order of garlic bread and two Diet Cokes?
  B: Absolutely. Can I have your delivery address please?
  A: It’s 452 Park Avenue, Apartment 21B.
  B: Great, your total will be $24.50. Is there anything else?
  A: No, that’s all. How long will the delivery take?
  B: It should take about 30-40 minutes. We’ll get it to you as soon as possible.
  A: Sounds good, thank you!

- 한글 해석:
  A: 안녕하세요, 배달로 피자 주문하고 싶습니다.
  B: 네, 무엇을 드릴까요?
  A: 대형 페페로니 피자에 치즈를 추가해 주세요.
  B: 사이드 메뉴나 음료는 필요하신가요?
  A: 예, 마늘빵 하나와 다이어트 콜라 두 개 주세요.
  B: 알겠습니다. 배달 주소 알려주시겠어요?
  A: 452 파크 애비뉴, 21B 아파트입니다.
  B: 좋습니다, 총 금액은 $24.50입니다. 다른 것 필요하신 건 없나요?
  A: 아니요, 그게 다입니다. 배달은 얼마나 걸리나요?
  B: 대략 30-40분 걸릴 예정입니다. 최대한 빨리 배달해 드리겠습니다.
  A: 좋습니다, 감사합니다!


In [25]:
# 완성된 chain을 실행하여 답변을 얻는다
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
   - Server: "Hello! Welcome to our restaurant. Are you ready to order or do you need a few more minutes?"
   - You: "Hi, I think I need a couple more minutes. Could I get a glass of water, please?"
   - Server: "Of course! I’ll bring your water right away. Just let me know when you’re ready to order."
   - You: "Thank you. Actually, can I ask about today’s specials?"
   - Server: "Today we have grilled salmon with a lemon butter sauce and a side of asparagus, or there’s a roasted chicken served with mashed potatoes and gravy."
   - You: "That sounds delicious. I’ll have the grilled salmon, please."
   - Server: "Great choice! How would you like the salmon cooked?"
   - You: "Could I have it cooked medium, please?"
   - Server: "Sure thing. Would you like anything to drink with your meal?"
   - You: "Yes, I’ll have a glass of white wine. Could you recommend one?"
   - Server: "Certainly! The Chardonnay pairs very well with our salmon. It’s light and refreshing."
   - You: "Tha

In [26]:
# 이번에는 question을 "미국에서 피자 주문"으로 설정한 것을 스트리밍으로 출력
answer1 = chain.stream({"question": "미국에서 피자 주문"})
stream_response(answer1)

#상황:
미국에서 피자 주문

#FORMAT:
- 영어 회화:
   **Customer:** Hi, I’d like to order a large pepperoni pizza for delivery, please.
   **Operator:** Sure thing! Can I get your address, please?
   **Customer:** It’s 124 Elm Street, Apartment 7, Springfield.
   **Operator:** Great! And would you like to add any sides or drinks to your order?
   **Customer:** Yes, let’s add a side of garlic knots and two cans of Coke.
   **Operator:** Perfect! That’s a large pepperoni pizza, garlic knots, and two Cokes. Will that be all?
   **Customer:** Yes, that’s it. How long will the delivery take?
   **Operator:** It should be there in about 30-45 minutes. The total comes to $28.50. Is that alright?
   **Customer:** That sounds good. Can I pay with a credit card?
   **Operator:** Of course. I can take your card information now.
   **Customer:** Sure, the number is 1234 5678 9012 3456, expiration date 04/24, and the CVV is 789.
   **Operator:** Thank you! Your order has been placed and should arrive within the ne